In [40]:
import numpy as np
import scipy as sp
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from skimage import data
from skimage import transform
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, plot_matches, SIFT

# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'

In [41]:
def triangulate_3D_point_DLT(P1, P2, img1_pts, img2_pts, print_svd=False):
    
    n = np.size(img1_pts,1)
    X = []

    for i in range(n):

        x1 = img1_pts[0,i]
        y1 = img1_pts[1,i]
        
        x2 = img2_pts[0,i]
        y2 = img2_pts[1,i]

        M = np.array([[P1[0,0]-x1*P1[2,0], P1[0,1]-x1*P1[2,1], P1[0,2]-x1*P1[2,2], P1[0,3]-x1*P1[2,3]],
                      [P1[1,0]-y1*P1[2,0], P1[1,1]-y1*P1[2,1], P1[1,2]-y1*P1[2,2], P1[1,3]-y1*P1[2,3]],
                      [P2[0,0]-x2*P2[2,0], P2[0,1]-x2*P2[2,1], P2[0,2]-x2*P2[2,2], P2[0,3]-x2*P2[2,3]],
                      [P2[1,0]-y2*P2[2,0], P2[1,1]-y2*P2[2,1], P2[1,2]-y2*P2[2,2], P2[1,3]-y2*P2[2,3]]])

        U, S, VT = np.linalg.svd(M, full_matrices=False)
        X.append(VT[-1,:])
        
        if print_svd:
            M_approx = U @ np.diag(S) @ VT
            v = VT[-1,:] # last row of VT because optimal v should be last column of V
            Mv = M @ v
            print('\n||Mv||:', (Mv @ Mv)**0.5)
            print('||v||^2:', v @ v)
            print('max{||M - M_approx||}:', np.max(np.abs(M - M_approx)))
            print('S:', S)

    X = np.stack(X,1)
    return X

In [42]:
def plot_image_points_projected_points_and_image(x_proj, x_img, img, path):

    fig = plt.figure(figsize=(17,8))

    plt.plot(x_img[0], x_img[1], 'D', color='cyan', ms=3, label='SIFT points')
    plt.plot(x_proj[0], x_proj[1], 's', color='red', ms=1.5, label='Projected points')

    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('equal')
    plt.gca().invert_yaxis()

    plt.legend(loc="lower right")
    # plt.aspect('equal')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    fig.savefig(path, dpi=300)

    plt.show()

In [43]:
a = np.array([True,True])
b = np.array([False,True])
a*b

array([False,  True])

In [44]:
def compute_average_error(x_proj, x_img):
    err = ((x_proj[0,:] - x_img[0,:])**2 + (x_proj[1,:] - x_img[1,:])**2)**0.5
    avg_err = np.sum(err) / np.size(err,0)
    return avg_err

In [45]:
def remove_error_points(x1_proj, x1_img, x2_proj, x2_img, X, err):

    x1_keep = (((x1_proj[0,:] - x1_img[0,:])**2 + (x1_proj[1,:] - x1_img[1,:])**2)**0.5 < err)
    x2_keep = (((x2_proj[0,:] - x2_img[0,:])**2 + (x2_proj[1,:] - x2_img[1,:])**2)**0.5 < err)
    x_keep = x1_keep * x2_keep

    x1_proj_keep = x1_proj[:,x_keep]
    x1_img_keep = x1_img[:,x_keep]

    x2_proj_keep = x2_proj[:,x_keep]
    x2_img_keep = x2_img[:,x_keep]

    X_keep = X[:,x_keep]
    
    return x1_proj_keep, x1_img_keep, x2_proj_keep, x2_img_keep, X_keep

In [46]:
def plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path, Xmodel, startind, endind):
    
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection='3d')

    ax.plot(X[0], X[1], X[2], '.', ms=1, color='magenta', label='3D points')
    
    for i in range(len(startind)):
        start = startind[i]-1
        end = endind[i]-1
        x = [Xmodel[0,start], Xmodel[0,end]]
        y = [Xmodel[1,start], Xmodel[1,end]]
        z = [Xmodel[2,start], Xmodel[2,end]]
        ax.plot(x, y, z, '-', color='orange')
    
    cv.plot_cameras_and_axes(ax, C_arr, axis_arr, s)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.axis('equal')
    ax.set_aspect('equal')

    plt.legend(loc="lower right")
    fig.savefig(path, dpi=300)

    plt.show()
    

In [53]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-2'
data = r'\A2data\data'
report = r'\report-images'

cube1_img = cv.load_image(path+data+'/cube1.jpg')
cube2_img = cv.load_image(path+data+'/cube2.jpg')

cube1_pts = cv.homogenize(np.load(path+data+'/cube1_pts.npy'), multi=True)
cube2_pts = cv.homogenize(np.load(path+data+'/cube2_pts.npy'), multi=True)

P1 = np.load(path+data+'/P1.npy')
P2 = np.load(path+data+'/P2.npy')

norm = True
if norm:
    K1, R1 = cv.rq(P1)
    K2, R2 = cv.rq(P2)

    K1_inv = np.linalg.inv(K1)
    K2_inv = np.linalg.inv(K2)

    P1 = cv.transform(K1_inv, P1)
    P2 = cv.transform(K2_inv, P2)

    cube1_pts = cv.transform_and_dehomogenize(K1_inv, cube1_pts)
    cube2_pts = cv.transform_and_dehomogenize(K2_inv, cube2_pts)

X = cv.dehomogenize(triangulate_3D_point_DLT(P1, P2, cube1_pts, cube2_pts))
cube1_proj = cv.transform_and_dehomogenize(P1, X)
cube2_proj = cv.transform_and_dehomogenize(P2, X)

if norm:
    cube1_pts = cv.transform_and_dehomogenize(K1, cube1_pts)
    cube2_pts = cv.transform_and_dehomogenize(K2, cube2_pts)

    cube1_proj = cv.transform_and_dehomogenize(K1, cube1_proj)
    cube2_proj = cv.transform_and_dehomogenize(K2, cube2_proj)

    avg_err_1 = compute_average_error(cube1_proj, cube1_pts)
    avg_err_2 = compute_average_error(cube2_proj, cube2_pts)
    print('\navg_err_1:', avg_err_1, '\navg_err_2:', avg_err_2)

pix_err = 3
print('Points before removal:',np.size(X,1))
cube1_proj, cube1_pts, cube2_proj, cube2_pts, X = remove_error_points(cube1_proj, cube1_pts, cube2_proj, cube2_pts, X, pix_err)
print('Points after removal:',np.size(X,1))

avg_err_1 = compute_average_error(cube1_proj, cube1_pts)
avg_err_2 = compute_average_error(cube2_proj, cube2_pts)
print('\navg_err_1:', avg_err_1, '\navg_err_2:', avg_err_2)

x_trans = np.array([[np.size(cube1_img,1)],[0],[0]])
cube_cat_img = np.concatenate((cube1_img, cube2_img), 1)

cube_proj = np.concatenate((cube1_proj, x_trans+cube2_proj), 1)
cube_pts = np.concatenate((cube1_pts, x_trans+cube2_pts), 1)

# plot_image_points_projected_points_and_image(cube_proj, cube_pts, cube_cat_img, path+report+'/E4_proj_pts_img_3.png')

Ps =np.array([P1,P2])
C_arr = np.array([cv.homogenize(cv.compute_camera_center(Ps[i])) for i in range(np.size(Ps,0))]).T
axis_arr = np.array([cv.homogenize(cv.compute_normalized_principal_axis(Ps[i])) for i in range(np.size(Ps,0))]).T

Xmodel = cv.convert_mat_to_np(path+data+'/compEx3data.mat', 'Xmodel')
startind = cv.convert_mat_to_np(path+data+'/compEx3data.mat', 'startind')
endind = cv.convert_mat_to_np(path+data+'/compEx3data.mat', 'endind') 

s = 30
# plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path+report+'/E4_3D.png', Xmodel, startind, endind)


# Norm
# avg_err_1: 186.7856654252198 
# avg_err_2: 116.39093336994448
# Points before removal: 6412
# Points after removal: 1926

# avg_err_1: 1.1899625386672275 
# avg_err_2: 1.2266875250340137


avg_err_1: 186.7856654252198 
avg_err_2: 116.39093336994448
Points before removal: 6412
Points after removal: 1926

avg_err_1: 1.1899625386672275 
avg_err_2: 1.2266875250340137
